### DS7337 NLP - HW 3
#### David Wei

# Homework 3

<u>**HW 3:**</u>

[book link](http://www.nltk.org/book/)

1. Compare your given name with your nickname (if you don’t have a nickname, invent one for this assignment) by answering the following questions:

    - What is the edit distance between your nickname and your given name?
    - What is the percentage string match between your nickname and your given name?
Show your work for both calculations.

2. Find a friend (or family member or classmate) who you know has read a certain book. Without your friend knowing, copy the first two sentences of that book. Now rewrite the words from those sentences, excluding stop words. Now tell your friend to guess which book the words are from by reading them just that list of words. Did you friend correctly guess the book on the first try? What did he or she guess? Explain why you think you friend either was or was not able to guess the book from hearing the list of words. 

3. Run one of the stemmers available in Python. Run the same two sentences from question 2 above through the stemmer and show the results. How many of the outputted stems are valid morphological roots of the corresponding words? Express this answer as a percentage.



In [1]:
import nltk
from urllib import request
import matplotlib as plt
import os
from nltk.tokenize import RegexpTokenizer
import numpy as np
from sklearn.preprocessing import minmax_scale
from IPython.display import Image
from IPython.core.display import HTML 
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation

## Q1, Pt 1 - Edit Distance of Name and Nickname

To begin, we will first compare my nickname, "DWei" which my family calls me and my given name, "David". Using the Levenshtein edit-distance between the nickname and given name, we'll look to find the number of characters that need to be substituted, inserted, or deleted, to transform s1 into s2.

Source: https://tedboy.github.io/nlps/generated/generated/nltk.edit_distance.html

In [2]:
nick_name = "DWei"
given_name = "David Wei"

edit_distance = nltk.edit_distance(nick_name, given_name, transpositions=False)
print('edit distance: ',edit_distance)

edit distance:  5


## Q1, Pt 2 - Percentage String Match of Name and Nickname

We will next calculate the percentage string match between the nickname and the given name. To do this, we will continue to utilize the Levenshtein Distance which is a metric to measure how apart two sequences of words are. The percentage string match will simply be the ratio between the 2 strings given their distance as shown in the equation below:

![title](https://i0.wp.com/predictivehacks.com/wp-content/uploads/2020/12/image-3.png?resize=422%2C158&ssl=1)

Source: https://www.datacamp.com/community/tutorials/fuzzy-string-python 

In [3]:
import numpy as np
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

In [4]:
# normalize name inputs by assigning lowercase value to them
distance = levenshtein_ratio_and_distance(nick_name.lower(), given_name.lower(), ratio_calc=True )
print(str(distance*100)+'%')

61.53846153846154%


We can see that my nickname and my given name are roughly only 62% similar. This could be that the last name portion of my nickname 'Wei' is also contained in the full given name. However, because the nickname abbreviates my given names first name it cannot find any way to transform it back which accounts for the the other 38%.  

## Q2 - Guessing text without stopwords

In [5]:
# import text
tale_of_2_cites = "https://www.gutenberg.org/files/98/98-h/98-h.htm"

# reads in electronic book
def getRaw(text):
    url = text
    response = request.urlopen(url)
    raw = response.read().decode('utf8')
    normalized_text = raw.lower()
    raw_character_count = len(raw)
    return raw, raw_character_count, normalized_text

# retrieving raw text from book
raw_text = getRaw(tale_of_2_cites)[0]

# return first 1 sentence locations
start_text = raw_text.find("It was the best of times, it was the worst of times")
end_text = raw_text.find("It was the year of Our Lord one thousand seven hundred and seventy-five")
print('Start of text: ',str(start_text))
print('End of text: ',str(end_text))

# extracting only the first 2 sentences and cleaning it
text = raw_text[start_text:end_text].replace('\r\n', '').replace('</p>','').replace('<p>', '')

# removing punctuation
tokenizer = nltk.RegexpTokenizer(r"\w+")
text_cleaned = tokenizer.tokenize(text)

# turning cleaned tokenized text back to string format
from nltk.tokenize.treebank import TreebankWordDetokenizer
text_nonalpha = TreebankWordDetokenizer().detokenize(text_cleaned)

Start of text:  8507
End of text:  9593


In [15]:
stop_words = set(stopwords.words('english'))
word_tokens = word_tokenize(text_nonalpha)
 
# # filtering out stopwords from text
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
        filtered_sentence.append(' ')
        full_filtered_sentece=''.join(filtered_sentence)

# return first 10 words from text with stop words removed
print('"'+str(full_filtered_sentece)+'"')

"It best times worst times age wisdom age foolishness epoch belief epoch incredulity season Light season Darkness spring hope winter despair everything us nothing us going direct Heaven going direct way mdash short period far like present period noisiest authorities insisted received good evil superlative degree comparison There king large jaw queen plain face throne England king large jaw queen fair face throne France In countries clearer crystal lords State preserves loaves fishes things general settled ever "


After filtering out all the stop words from the the original text, I had my girlfriend read the paragraph above without any prior context other than to guess what the book was. Knowing that she had read the classic Charle's Dicken book 'A Tale of Two Cities' before in her childhood, I was surprised when she off the bat did not know what book she was reading from. My guess is that inflection matters and the stop words add a sort of linguistic poetry as the words roll of the tongue that helps readers recall information as they recite it. In this case of the famous book intro "It **was** the best **of** times, it **was** the worst **of** times", both 'was' and 'of' gave the text a sort of poetic feel to it and without it, would be hard to distinguish. 

## Q3 - Percentage Match of text using Stemming

Stemming is the process of reducing inflection in words to their root forms even if the stem itself is not a valid word in the language. To find how similar the outputted stems are to their valid morphological root, we will reruse the Levenshtein edit-distance ratio to compare the before and after similarity of our text.

In [21]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

def stemText(text):
    stem_text=[]
    for word in text:
        stem_text.append(ps.stem(word))
        stem_text.append(" ")
    return "".join(stem_text)

stemmed_text_nostop = stemText(filtered_sentence)
stemmed_text = stemText(word_tokens)

In [22]:
# normalize name inputs by assigning lowercase value to them
distance_stemmed_nostop = levenshtein_ratio_and_distance(stemmed_text_nostop.lower(), tale_of_2_cites.lower(), ratio_calc=True )
distance_stemmed = levenshtein_ratio_and_distance(stemmed_text.lower(), tale_of_2_cites.lower(), ratio_calc=True )
print('% match with stopwords removed:',str(distance_stemmed_nostop*100)+'%')
print('% match with stemming only:',str(distance_stemmed*100)+'%')

% match with stopwords removed: 7.2727272727272725%
% match with stemming only: 6.23608017817372%


To do our stemming, we will utilize Porter's Stemmer, which comparitively to it's other stemming counterparts (ex. Snowball, Lancaster) provides the least aggresive algorithm in it's stemming approach. Using the Porter's Stemmer as our 'best case' scenario stemming, we then ran the stemmer through 2 text groups: one with stop words removed and the other with only the original tokenized text stemmed. We can see that based on our results, the overall similarity between our stemmed text and it's morphological original text was drastically different. However, we also observed that the stemmed text with it's stopwords removed resulted in a slightly higher percentage match, this could be due to how including stop words resulted in non-valid terms and by removing them entirely more matching.